In [1]:
import numpy as np
import pandas as pd
from binerizer import *
from DNFRuleModel import DNFRuleModel
from Classifier import Classifier
from sklearn.model_selection import train_test_split
import random
import time
import json
import copy

In [2]:
heart = pd.read_csv('data/heart.csv')
heart_bin = binerizeData(heart, verbose = False)

heart_bin[heart_bin.columns[0]] = heart_bin['Y']

heart_X =heart_bin.to_numpy()[:,0:(heart_bin.shape[1]-1)]
heart_Y = heart_bin.to_numpy()[:,heart_bin.shape[1]-1]

rule = np.zeros(heart_bin.shape[1]-1)
rule[0] = 1
rule_set = [rule, np.zeros(heart_bin.shape[1]-1)]
rule_set

args = {
                'ruleGenerator': 'DNF_IP',
                'numRulesToReturn': 100,
                'epsilon': 1,
                'fairness_module': 'EqOfOp',
                'group': heart_bin['X1_45.0*'].to_numpy().astype(np.bool)
            }

heart_classif = Classifier(heart_X, heart_Y, args)
heart_classif.fit(initial_rules = rule_set, verbose = True, colGen = False, rule_filter = True)

EqOfOp
Using DNF_IP
Using license file /Users/connorlawless/gurobi.lic
Academic license - for non-commercial use only
Solving final master problem to integer optimality
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 141 rows, 140 columns and 692 nonzeros
Model fingerprint: 0xb91c3c9d
Coefficient statistics:
  Matrix range     [8e-03, 2e+00]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 141 rows and 140 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  0.000000000e+00
eps[0] 0
eps[1] 0
eps[2] 0
eps[3] 0
eps[4] 0
eps[5] 0
eps[6] 0
eps[7] 0
eps[8] 0
eps[9] 0
eps[10] 0
eps[11] 0
eps[12] 0
eps[13] 0
eps[14] 0
eps[15] 

In [3]:
heart = pd.read_csv('data/heart.csv')
heart_bin = binerizeData(heart, verbose = False)

heart_X =heart_bin.to_numpy()[:,0:(heart_bin.shape[1]-1)]
heart_Y = heart_bin.to_numpy()[:,heart_bin.shape[1]-1]

print(heart)
complex_grid = [1, 5, 10, 15, 20, 25, 40, 60, 80, 100, 120]
price_limit = 45
train_limit = 300

accuracies = []
accuracies_full = []

for c in [14]:
    accuracy = []
    args = {'ruleComplexity': c}
    for i in range(10):
        X_train, X_test, y_train, y_test = train_test_split(heart_X, heart_Y, test_size=0.2)
        print(y_test)
        heart_classif = Classifier(X_train, y_train, args)
        heart_classif.fit(verbose = False, timeLimit = train_limit, timeLimitPricing = price_limit)
        acc = sum(heart_classif.predict(X_test) == y_test)/len(y_test)
        accuracy.append(acc)
    accuracies.append(np.mean(accuracy))
    accuracies_full.append(accuracy)

     X1  X2  X3   X4   X5  X6  X7   X8  X9  X10  X11  X12  X13      Y
0    63   1   1  145  233   1   2  150   0  2.3    3  0.0  6.0  False
1    67   1   4  160  286   0   2  108   1  1.5    2  3.0  3.0   True
2    67   1   4  120  229   0   2  129   1  2.6    2  2.0  7.0   True
3    37   1   3  130  250   0   0  187   0  3.5    3  0.0  3.0  False
4    41   0   2  130  204   0   2  172   0  1.4    1  0.0  3.0  False
..   ..  ..  ..  ...  ...  ..  ..  ...  ..  ...  ...  ...  ...    ...
294  57   0   4  140  241   0   0  123   1  0.2    2  0.0  7.0   True
295  45   1   1  110  264   0   0  132   0  1.2    2  0.0  7.0   True
296  68   1   4  144  193   1   0  141   0  3.4    2  2.0  7.0   True
297  57   1   4  130  131   0   0  115   1  1.2    2  1.0  7.0   True
298  57   0   2  130  236   0   2  174   0  0.0    2  1.0  3.0   True

[299 rows x 14 columns]
[False False False  True False False False  True False False  True False
 False False  True False False False False False  True False  

KeyboardInterrupt: 

In [ ]:
myDict = {'hi': {'hola':'spanish', 'bonjour':'french'}}
newDict = copy.deepcopy(myDict)
del newDict['hi']['hola']
newDict

In [ ]:
#load rules
with open('results/FairnessRuns-Eps:0.01_rules.txt') as json_file:
    rules = np.array(json.load(json_file))

In [ ]:
dataInfo = {
    'file': 'data/german.csv',
    'groupCol': 'gender'
}
globalArgs = {
    'num_splits': 10,
}

tests = []
for eps in [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.07, 0.1, 0.2, 0.5, 1]:
    test_params = {
            'price_limit': 10,
            'train_limit': 10,
            'fixed_model_params': {
                'ruleGenerator': 'DNF_IP',
                'numRulesToReturn': 100,
                'epsilon': eps,
                'fairness_module': 'EqOfOp'
            },
            'num_hp_splits': 3,
            'name': 'FairnessRuns-Eps:'+str(eps),
            'hyper_paramaters': {
                'ruleComplexity': [5,10,17, 20]
            }
        }
    tests.append(test_params)

results, _ = run_fairness_curve(tests, globalArgs, dataInfo, rules, verbose = False)

In [ ]:
summary_results = []
for res in results:
    summary_results.append([res.name.split(':')[-1],  res.res['mean_accuracy'], res.res['std_accuracy']])
pd.DataFrame(summary_results, columns = ['epsilon', 'mean_accuracy',  'std_accuracy']).to_csv('results/german.csv')